In [1]:
import yahoo_fin.stock_info as si

In [2]:
from pandas import read_html as rh
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

### Function for getting fundamental data from Yahoo Finance

In [3]:
def get_index(num):
    idx = []
    for i in range(len(num)):
        if num[i].get_text() == 'Trailing P/E ':
            idx.append(i)
        elif num[i].get_text() == 'Forward P/E ':
            idx.append(i)
        elif num[i].get_text() == 'Price/Book (mrq)':
            idx.append(i)
        elif num[i].get_text() == 'Enterprise Value/Revenue ':
            idx.append(i)
        elif num[i].get_text() == 'Enterprise Value/EBITDA ':
            idx.append(i)
    return idx


In [94]:
get_index(num)

[14, 21, 42, 49, 56]

In [45]:
def get_results(company_list):
    #Get X1, X2, X3 for company in company list
    result = {'company':[],'delta_PE':[], 'delta_PB':[], 'delta_EV':[]}
    for company in company_list:
        r = requests.get(f'https://finance.yahoo.com/quote/{company}/key-statistics?p={company}', headers = {'User-Agent':'Mozilla/5.0'})
        soup = bs(r.content, 'lxml')
        table = soup.find_all('table')
        num = table[0].find_all('td')
        result['company'].append(company)
        idx = get_index(num)
        if num[idx[0]+2].get_text() == 'N/A' or num[idx[1]-1].get_text() == 'N/A':
            result['delta_PE'].append('N/A')
        else:
            if num[idx[0]+2].get_text()[-1] == 'k':
                p1 = float(num[idx[0]+2].get_text()[:-1])*1000
            else:
                p1 = float(num[idx[0]+2].get_text())
            if num[idx[1]-1].get_text()[-1] == 'k':
                p2 = float(num[idx[1]-1].get_text()[:-1])*1000
            else:
                p2 = float(num[idx[1]-1].get_text())
            
            result['delta_PE'].append(p1-p2)
            
        if num[idx[2]+2].get_text() == 'N/A' or num[idx[3]-1].get_text() == 'N/A':
            result['delta_PB'].append('N/A')
        else:
            result['delta_PB'].append(float(num[idx[2]+2].get_text())-float(num[idx[3]-1].get_text()))  
            
        if num[idx[4]+2].get_text() == 'N/A' or num[-1].get_text() == 'N/A':
            result['delta_EV'].append('N/A')        
        else:
            result['delta_EV'].append(float(num[idx[4]+2].get_text())-float(num[-1].get_text()))
            
        print('finished %s'%company)
    return pd.DataFrame(result)

In [55]:
company_list = ['T','AVGO','MA','FB','ACN','UPS','KO','GOOG','MRK','PM','ABT','DHR','DIS','XOM','LOW','GOOGL','UNH','CSCO'
               ,'CVX','AAPL','MSFT','TXN','WFC','PG','NVDA','TMO','AMZN','NFLX','LLY','TSLA','VZ','JPM','BAC','BRK-B',
               'PFE','QCOM','ABBV','ADBE','PEP','NKE']

In [56]:
df = get_results(company_list)

finished T
finished AVGO
finished MA
finished FB
finished ACN
finished UPS
finished KO
finished GOOG
finished MRK
finished PM
finished ABT
finished DHR
finished DIS
finished XOM
finished LOW
finished GOOGL
finished UNH
finished CSCO
finished CVX
finished AAPL
finished MSFT
finished TXN
finished WFC
finished PG
finished NVDA
finished TMO
finished AMZN
finished NFLX
finished LLY
finished TSLA
finished VZ
finished JPM
finished BAC
finished BRK-B
finished PFE
finished QCOM
finished ABBV
finished ADBE
finished PEP
finished NKE


In [96]:
df

,company,delta_PE,delta_PB,delta_EV
0,T,170.18,-0.1,82.18
1,AVGO,-31.79,1.29,-1.66
2,MA,-9.39,-8.77,-32.31
3,FB,-7.05,0.41,10.89
4,ACN,7.45,2.29,13.79
5,UPS,-2.63,-10.57,89.73
6,KO,-0.35,-2.5,-16.44
7,GOOG,-5.99,2.27,16.55
8,MRK,4.96,-1.68,314.91
9,PM,-0.04,N/A,-0.22


### Function for getting return data, thereby labeling companies as outperform or underperform

In [59]:
from yahoo_fin.stock_info import get_data

In [80]:
def label_performance(company_list):
    spy_start = get_data('spy', start_date = '01/03/2022' , end_date = '01/04/2022', index_as_date = True, interval = '1d').iloc[0,0]
    spy_end = get_data('spy', start_date = '01/31/2022' , end_date = '02/01/2022', index_as_date = True, interval = '1d').iloc[0,3]
    spy_ret = (spy_end - spy_start)/spy_start*100
    print('spy return is %f percent'%spy_ret)
    result = {'company':[],'return':[], 'label':[]}
    for company in company_list:
        company_start = get_data(company, start_date = '01/03/2022' , end_date = '01/04/2022', index_as_date = True, interval = '1d').iloc[0,0]
        company_end = get_data(company, start_date = '01/31/2022' , end_date = '02/01/2022', index_as_date = True, interval = '1d').iloc[0,3]
        company_ret = (company_end - company_start)/company_start*100
        print('%s return is %f percent'%(company, company_ret))
        result['company'].append(company)
        result['return'].append(company_ret)
        if company_ret > spy_ret:
            result['label'].append(1)
        else:
            result['label'].append(0)
    return pd.DataFrame(result)

In [85]:
perf = label_performance(company_list)

spy return is -5.540622 percent
T return is 2.905566 percent
AVGO return is -12.072278 percent
MA return is 7.390421 percent
FB return is -7.401708 percent
ACN return is -14.730144 percent
UPS return is -5.795475 percent
KO return is 3.723221 percent
GOOG return is -6.075080 percent
MRK return is 6.398539 percent
PM return is 8.388660 percent
ABT return is -8.584949 percent
DHR return is -12.634503 percent
DIS return is -8.252583 percent
XOM return is 24.036573 percent
LOW return is -8.429786 percent
GOOGL return is -6.722623 percent
UNH return is -5.485999 percent
CSCO return is -11.494441 percent
CVX return is 11.846367 percent
AAPL return is -1.715123 percent
MSFT return is -7.267033 percent
TXN return is -4.941215 percent
WFC return is 9.952992 percent
PG return is -0.766903 percent
NVDA return is -17.873552 percent
TMO return is -12.321453 percent
AMZN return is -10.729037 percent
NFLX return is -29.469456 percent
LLY return is -10.575418 percent
TSLA return is -18.386411 percent


In [86]:
perf

,company,return,label
0,T,2.905566,1
1,AVGO,-12.072278,0
2,MA,7.390421,1
3,FB,-7.401708,0
4,ACN,-14.730144,0
5,UPS,-5.795475,0
6,KO,3.723221,1
7,GOOG,-6.075080,0
8,MRK,6.398539,1
9,PM,8.388660,1


In [89]:
perf.to_excel('perf.xlsx')

In [90]:
company_list = ['JNJ','CRM','INTC','COST','CMCSA','MCD','HD','WMT','V','INTU']

In [91]:
df2 = get_results(company_list)

finished JNJ
finished CRM
finished INTC
finished COST
finished CMCSA
finished MCD
finished HD
finished WMT
finished V
finished INTU


In [92]:
df2

,company,delta_PE,delta_PB,delta_EV
0,JNJ,0.82,-0.02,-23.56
1,CRM,61.64,0.17,107.49
2,INTC,0.21,-0.38,4.88
3,COST,4.43,4.13,17.97
4,CMCSA,-7.32,-0.41,-1.44
5,MCD,-5.12,N/A,19.02
6,HD,2.75,-2.08,-0.88
7,WMT,21.70,0.28,-101.80
8,V,-6.24,-1.9,-29.59
9,INTU,34.31,-1.54,-325.93


In [93]:
df2.to_excel('confirm.xlsx')

In [94]:
perf2 = label_performance(company_list)

spy return is -5.540622 percent
JNJ return is 1.222012 percent
CRM return is -8.776123 percent
INTC return is -5.479190 percent
COST return is -10.601211 percent
CMCSA return is 0.020012 percent
MCD return is -3.725548 percent
HD return is -11.904361 percent
WMT return is -2.909724 percent
V return is 3.976643 percent
INTU return is -13.249386 percent


In [95]:
perf2.to_excel('perf2.xlsx')